In this notebook we'll describe the basics of the Element RNN library, which offers implementations and variations of LSTM and GRU recurrent networks. You should check out the documentation and examples at https://github.com/Element-Research/rnn

## Review
Recall that a recurrent neural network maps a sequence of vectors $\mathbf{x}_1, \ldots, \mathbf{x}_n$ to a sequence of vectors $\mathbf{s}_1, \ldots, \mathbf{s}_n$ using the recurrence

\begin{align*}
\mathbf{s}_{i} =  R(\mathbf{x}_{i}, \mathbf{s}_{i-1}; \mathbf{\theta}),
\end{align*}

where $R$ is a function parameterized by $\mathbf{\theta}$, and we define $\mathbf{s}_0$ as some initial vector (such as a vector of all zeros).

### N.B. For the first part of this notebook we will deal only with "acceptor" RNNs
* That is, we'll assume we only use the final state $\mathbf{s}_n$ for making predictions


## Element RNN Basics
At the heart of the Element RNN library is the abstract class 'AbstractRecurrent', which is designed to allow calling :forward() on each element $\mathbf{x}_i$ of a sequence (in turn), with the abstract class keeping track of the $\mathbf{s}_i$ for you. Consider the following example.

In [2]:
require 'rnn' -- this imports 'nn' as well, and adds 'rnn' objects to the 'nn' namespace

lstm = nn.LSTM(5, 5) -- inherits from AbstractRecurrent, and expects inputs in R^5 and produces outputs in R^5
data = torch.randn(3, 5) -- a sequence of 3 random vectors, each in R^5
outputs = torch.zeros(3, 5)

for i = 1, data:size(1) do
    outputs[i] = lstm:forward(data[i]) -- note that we don't need to keep track of the s_i
end

print(outputs)

 0.0055 -0.0308 -0.0706 -0.2473 -0.0432
 0.0298 -0.0373  0.0276 -0.4038 -0.1443
-0.1507 -0.0742  0.1933 -0.3205 -0.2959
[torch.DoubleTensor of size 3x5]



Above, the lstm is able to compute :forward() at each step by keeping track of its states internally. As we discussed in lecture, in an LSTM the states $\mathbf{s}_i$ comprise the 'hidden state' $\mathbf{h}_i$ as well as the 'cell' $\mathbf{c}_i$. In the RNN package's terminology, the $\mathbf{h}_i$ are known as 'outputs', and the $\mathbf{c}_i$ as 'cells', and these are stored internally in the nn.LSTM object, as tables:

In [2]:
print(lstm.outputs)
print(lstm.cells)

{
  1 : DoubleTensor - size: 5
  2 : DoubleTensor - size: 5
  3 : DoubleTensor - size: 5
}
{
  1 : DoubleTensor - size: 5
  2 : DoubleTensor - size: 5
  3 : DoubleTensor - size: 5
}


We can see that lstm.outputs are the same as the outputs tensor we stored manually

In [3]:
print(lstm.outputs[1])
print(lstm.outputs[2])
print(lstm.outputs[3])

 0.0354
-0.2311
-0.0782
-0.0994
 0.1091
[torch.DoubleTensor of size 5]

-0.0087
-0.1867
-0.1211
-0.2268
 0.2223
[torch.DoubleTensor of size 5]

 0.0590
 0.1845
-0.0216
-0.2605
 0.1665
[torch.DoubleTensor of size 5]



So, the first thing the RNN library gives us is implementations of many of the $R$ functions we're interested in using, such as LSTMs, and GRUs. However, it does much more!

## BPTT

To do backpropagation (through time) correctly, we would technically need to loop backwards over the input sequence, as in the following pseudocode:

In [ ]:
-- note this doesn't work! just supposed to convey how you might have to implement this...
dLdh_i = gradOutForFinalH()
dLdc_i = gradOutForFinalC()

for i = data:size(1), 1, -1 do
    dLdh_iminus1, dLdc_iminus1 = lstm:backward(data[i], {dLdh_i, dLdc_i})
    dLdh_i, dLdc_i = dLdh_iminus1, dLdc_iminus1
end

Fortunately, however, the Element RNN library can do this for us, using its nn.Sequencer objects!

## Sequencers

An nn.Sequencer transforms a module (such as one inheriting from AbstractRecurrent) into a module that can call :forward() on an entire sequence, and :backward() on the entire sequence, thus abstracting away the looping required for backpropagation (through time). 

In particular, sequencers expect a **table** as input, and also output a table. Let's use an nn.Sequencer on an lstm:

In [16]:
lstm = nn.LSTM(5, 5)
seq_lstm = nn.Sequencer(lstm)
inp_table = torch.split(data, 1) -- make a table from our sequence of 3 vectors
out_table = seq_lstm:forward(inp_table)
print(out_table)

{
  1 : DoubleTensor - size: 1x5
  2 : DoubleTensor - size: 1x5
  3 : DoubleTensor - size: 1x5
}


For calling :backward(), an nn.Sequencer expects gradOutput in the same shape as its input, just as every other nn module does. In this case, then, an nn.Sequencer expects gradOutput to be table with an entry for each time step. In particular, gradOutput[i] should contain:

\begin{align*}
\frac{\partial \text{ loss at timestep } i}{\partial \mathbf{h}_i}
\end{align*}

Since for now we're dealing only with an "acceptor" RNN, there is only loss at the final timestep, and so gradOutput[i] is going to be all zeros, when $i < n$, as follows:

In [17]:
gradOutput = torch.split(torch.zeros(3, 5), 1)
-- randomly set final gradOutput
gradOutput[#gradOutput] = torch.randn(5) -- note that ordinarily you'd get this from a criterion
-- now we can BPTT with a single call!
seq_lstm:backward(inp_table, gradOutput)

## Avoiding Tables
Since your data will generally not be in tables (but in tensors), it's common to add additional layers to your network to map from tables to tensors and back. For instance, we can create an lstm that takes in a tensor (rather than a table), by using an nn.SplitTable

In [12]:
seq_lstm2 = nn.Sequential():add(nn.SplitTable(1)):add(nn.Sequencer(nn.LSTM(5, 5)))
print(seq_lstm2:forward(data))

{
  1 : DoubleTensor - size: 5
  2 : DoubleTensor - size: 5
  3 : DoubleTensor - size: 5
}


In an acceptor RNN we only care about the last state, so we can make our final layer a SelectTable (which also simplifies calling :backward(), since it implicitly passes back zeroes for all but the selected table index)

In [13]:
seq_lstm3 = seq_lstm2:clone()
seq_lstm3:add(nn.SelectTable(-1)) -- select the last element in the output table

print(seq_lstm3:forward(data))
gradOutFinal = gradOutput[#gradOutput] -- note that gradOutFinal is just a tensor
seq_lstm3:backward(data, gradOutFinal)

 0.1290
-0.0619
 0.1116
-0.0100
-0.0775
[torch.DoubleTensor of size 5]



If you cared about more than the last state of your LSTM, you could add an nn.JoinTable to your network after the Sequencer, which would give a tensor as output.

## A More Realistic Example
To make what we've seen so far a little more concrete, let's consider training an LSTM to predict whether song lyric 5-grams from 2012 are from a timeless masterpiece or not. Here's some data:

In [12]:
V = {["I"]=1, ["you"]=2, ["the"]=3, ["this"]=4, ["to"]=5, ["fire"]=6, ["Hey"]=7, ["is"]=8, 
    ["just"]=9, ["zee"]=10, ["And"]=11, ["just"]=12, ["rain"]=13, ["cray"]=14, ["met"]=15, ["Set"]=16}

-- get indices of words in each 5-gram
songData = torch.LongTensor({ { V["Hey"],   V["I"],      V["just"],  V["met"],    V["you"] },
                              { V["And"],   V["this"],   V["is"],    V["cray"],   V["zee"] },
                              { V["Set"],   V["fire"],   V["to"],    V["the"],    V["rain"] } })

masterpieceOrNot = torch.Tensor({{1},   -- #carlyrae4ever   
                                 {1},
                                 {0}}) 

print(songData)
-- we'll use a LookupTable to map word indices into vectors in R^6
vocabSize = 16
embedDim = 6
LT = nn.LookupTable(vocabSize, embedDim)

-- Using a Sequencer, let's make an LSTM that consumes a sequence of song-word embeddings
songLSTM = nn.Sequential()
songLSTM:add(LT) -- for a single sequence, will return a sequence-length x embedDim tensor
songLSTM:add(nn.SplitTable(1)) -- splits tensor into a sequence-length table containing vectors of size embedDim
songLSTM:add(nn.Sequencer(nn.LSTM(embedDim, embedDim)))
songLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
songLSTM:add(nn.Linear(embedDim, 1)) -- map last state to a score for classification
songLSTM:add(nn.Sigmoid()) -- convert score to a probability
firstSongPred = songLSTM:forward(songData[1])
print(firstSongPred)

  7   1  12  15   2
 11   4   8  14  10
 16   6   5   3  13
[torch.LongTensor of size 3x5]



 0.4135
[torch.DoubleTensor of size 1]



In [14]:
-- we can then use a simple BCE Criterion for backprop
bceCrit = nn.BCECriterion()
loss = bceCrit:forward(firstSongPred, masterpieceOrNot[1])
dLdPred = bceCrit:backward(firstSongPred, masterpieceOrNot[1])
songLSTM:backward(songData[1], dLdPred)

## Batching
In order to make RNNs fast, it is important to batch. When batching with an Element RNN, time-steps continue to be represented as indices in a table, but this time each element in the table is a **matrix** rather than a vector. In particular, batching occurs along the first dimension (as usual), as in the following example:

In [15]:
-- data representing a sequence of length 3, vectors in R^5, and batch-size of 2
batchSequenceDataTbl = {torch.randn(2, 5), torch.randn(2, 5), torch.randn(2, 5)}
print(batchSequenceDataTbl)
-- do a batched :forward() call
print(nn.Sequencer(nn.LSTM(5, 5)):forward(batchSequenceDataTbl))

{
  1 : DoubleTensor - size: 2x5
  2 : DoubleTensor - size: 2x5
  3 : DoubleTensor - size: 2x5
}


{
  1 : DoubleTensor - size: 2x5
  2 : DoubleTensor - size: 2x5
  3 : DoubleTensor - size: 2x5
}


As expected, gradOutput will also now be a sequence-length table of tensors that are batch_size x vector_size.

Now let's modify our song LSTM to consume data in batches:

In [22]:
-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
songDataT = songData:t()
batchSongLSTM = nn.Sequential()
batchSongLSTM:add(LT) -- will return a sequence-length x batch-size x embedDim tensor
batchSongLSTM:add(nn.SplitTable(1, 3)) -- splits into a sequence-length table with batch-size x embedDim entries
print(batchSongLSTM:forward(songDataT)) -- sanity check
-- now let's add the LSTM stuff
batchSongLSTM:add(nn.Sequencer(nn.LSTM(embedDim, embedDim)))
batchSongLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
batchSongLSTM:add(nn.Linear(embedDim, 1)) -- map last state to a score for classification
batchSongLSTM:add(nn.Sigmoid()) -- convert score to a probability
songPreds = batchSongLSTM:forward(songDataT)
print(songPreds)

{
  1 : DoubleTensor - size: 3x6
  2 : DoubleTensor - size: 3x6
  3 : DoubleTensor - size: 3x6
  4 : DoubleTensor - size: 3x6
  5 : DoubleTensor - size: 3x6
}


 0.5297
 0.5274
 0.5376
[torch.DoubleTensor of size 3x1]



In [23]:
-- we can now call :backward() as follows
loss = bceCrit:forward(songPreds, masterpieceOrNot)
dLdPreds = bceCrit:backward(songPreds, masterpieceOrNot)
batchSongLSTM:backward(songDataT, dLdPreds)

## Stacking RNNs

An additional benefit of the nn.Sequencer decorator is that it allows for easily stacking RNNs. When RNNs are stacked, the hidden state of the $i$'th time-step at the $l$'th layer is calculated using the hidden-state at the $i$'th time-step from $l-1$'th layer (i.e., the previous layer) as input, and the hidden-state at the $i-1$'th time-step from the $l$'th layer (i.e., the current layer) as the previous hidden state. Formally, we use the following recurrence

\begin{align*}
\mathbf{s}_{i}^{(l)} =  R(\mathbf{s}_{i}^{(l-1)}, \mathbf{s}_{i-1}^{(l)}; \mathbf{\theta}),
\end{align*}

where we must define initial vectors $\mathbf{s}_{0}^{(l)}$ for all layers $l$, and where $\mathbf{s}_{i}^{(0)} = \mathbf{x}_i$. That is, we consider the first layer to just be the input.

Let's see this in action.

In [24]:
stackedSongLSTM = nn.Sequential():add(LT):add(nn.SplitTable(1, 3))
stackedSongLSTM:add(nn.Sequencer(nn.LSTM(embedDim, embedDim))) -- add first layer
stackedSongLSTM:add(nn.Sequencer(nn.LSTM(embedDim, embedDim))) -- add second layer
print(stackedSongLSTM:forward(songDataT))

{
  1 : DoubleTensor - size: 3x6


  2 : DoubleTensor - size: 3x6
  3 : DoubleTensor - size: 3x6
  4 : DoubleTensor - size: 3x6
  5 : DoubleTensor - size: 3x6
}


In [25]:
-- let's make sure the recurrences happened as we expect
-- as a sanity check, let's print out the embeddings we sent into the lstm
print(LT:forward(songDataT))

(1,.,.) = 
 -0.7782 -1.2286  2.9292  0.6722 -1.0408 -1.4542
 -0.3506  0.6933  0.8812 -1.3697  0.5718  0.4102
 -0.9739 -0.3607 -2.6507 -0.5043 -0.1311 -0.9818

(2,.,.) = 
 -0.3579 -0.1492 -0.9239  0.1417 -1.2863 -0.3431
  2.0345 -0.5956  0.0774 -1.3960  0.7023 -1.9603
  0.7626 -1.9837  1.0503  0.5464 -1.4626 -0.2216

(3,.,.) = 
  0.6113  0.6440 -0.5835  1.5422  0.3331 -2.2186
  1.2035 -0.9474 -1.3586  0.1969  0.7381 -0.3102
  2.2829 -0.3115  0.7283 -0.4139  0.7639  0.6462

(4,.,.) = 
 -0.1971  0.2911 -0.4451 -1.3908 -2.1680  0.2561
  0.6389  0.1959  0.0224  0.2286  0.0596 -0.0350
  0.2773  0.7044  0.2160 -0.3286 -0.8221 -0.4745

(5,.,.) = 
  0.8532  0.5131 -0.7980  0.9659  1.1635  0.7443
  0.0968  0.5171 -0.7159 -0.0309 -0.7122 -0.1634
 -0.7714  1.2996 -1.1945  0.3587  0.8112  0.6116
[torch.DoubleTensor of size 5x3x6]




In [32]:
-- now let's look at the first layer LSTM's input at the third time-step; should match 3rd thing in the above!
firstLayerLSTM = stackedSongLSTM:get(3):get(1) -- Sequencer was 3rd thing added, and its first child is the LSTM
print(firstLayerLSTM.inputs[3])

 0.6113  0.6440 -0.5835  1.5422  0.3331 -2.2186
 1.2035 -0.9474 -1.3586  0.1969  0.7381 -0.3102
 2.2829 -0.3115  0.7283 -0.4139  0.7639  0.6462
[torch.DoubleTensor of size 3x6]



In [33]:
-- now let's look at the first layer LSTM's output at the 3rd time-step
print(firstLayerLSTM.outputs[3])

-0.0115 -0.0389  0.4011  0.3186 -0.3585 -0.1557
 0.0137 -0.2666  0.3196  0.1783  0.1917 -0.1368
-0.0905 -0.2193  0.2826  0.1088  0.0435  0.0217
[torch.DoubleTensor of size 3x6]




In [34]:
-- let's now examine the second layer LSTM and its input
secondLayerLSTM = stackedSongLSTM:get(4):get(1)
print(secondLayerLSTM.inputs[3]) -- should match the OUTPUT of firstLayerLSTM at 3rd timestep

-0.0115 -0.0389  0.4011  0.3186 -0.3585 -0.1557
 0.0137 -0.2666  0.3196  0.1783  0.1917 -0.1368
-0.0905 -0.2193  0.2826  0.1088  0.0435  0.0217
[torch.DoubleTensor of size 3x6]



## Remembering and Forgetting
When training an RNN, the training-loop usually looks like the following (in pseudocode):

In [ ]:
while stillInEpoch do
    batch = next batch of sequence-length x batch-size inputs
    lstm:forward(batch)
    gradOuts = gradOutput for each time-step (for each thing in the batch)
    lstm:backward(batch, gradOuts)
end

The sequence-length we choose to predict (and backprop through), however, is often shorter than the true length of the naturally occurring sequence it participates in. For example, we might imagine trying to train on 5-word windows of entire songs. In this case, we might arrange our data as follows:

 Sequence1    | Sequence2    
 :------- | :-------  
 Hey       | Set    
 I       | fire    
 just    | to      
 met    | the      
 **you**    | **rain**      
 And | Watched
 this       | it    
 is      | pour   
 crazy    | as      
 **but**    | **I**     
 ...    | ...      
 
where every 5th word is bolded.
 
Suppose now that we've just predicted and backprop'd through the first batch (of size 2), which consists of "Hey I just met you" and "Set fire to the rain." The second batch will then consist of "And this is crazy but" and "Watched it pour as I," respectively. When we start to predict on the first words of the second batch (namely, "And" and "Watched"), however, do we want to compute $\mathbf{s}_{And}$ and $\mathbf{s}_{Watched}$ using the final hidden states from the *previous* batch (namely, $\mathbf{s}_{you}$ and $\mathbf{s}_{rain}$) as our $\mathbf{s}_{i-1}$, or do we want to treat $\mathbf{s}_{And}$ and $\mathbf{s}_{Watched}$ as though they begin their own sequences, and simply use $\mathbf{s}_0$ as their respective $\mathbf{s}_{i-1}$? 

The answer may depend on your application, but often (such as in language modeling), we choose a relatively short sequence-length for convenience, even though we are interested in modeling the much longer sequence as a whole. In this case, it might make sense to "remember" the final state from the previous batch. You can control this behaviour with a Sequencer's :remember() and :forget() functions, as follows: 

In [41]:
-- let's make another LSTM
rememberLSTM = nn.Sequential():add(LT):add(nn.SplitTable(1, 3))
seqLSTM = nn.Sequencer(nn.LSTM(embedDim, embedDim))
-- possible arguments for :remember() are 'eval', 'train', 'both', or 'neither', which tells it whether to remember
-- only during evaluation (but not training), only during training, both or neither
seqLSTM:remember('both')  -- :remember() typically needs to only be called once
rememberLSTM:add(seqLSTM)

--since we're remembering, we expect inputting the same sequence twice in a row to give us different outputs
-- (since the first time the pre-first state is the zero vector, and the second it's the end of the sequence)
print(rememberLSTM:forward(songDataT)[5]) -- printing out just the final time-step

-- let's do it again with the same sequence!
print(rememberLSTM:forward(songDataT)[5]) -- printing out just the final time-step

 0.0631 -0.1597 -0.0998  0.0124  0.0474 -0.0257
-0.0005 -0.0683 -0.0124  0.0068 -0.0066 -0.0748
 0.1229  0.0247 -0.0984  0.0828  0.1579 -0.1836
[torch.DoubleTensor of size 3x6]



 0.0649 -0.1577 -0.1119  0.0099  0.0480 -0.0256
-0.0021 -0.0718 -0.0122  0.0051 -0.0027 -0.0724
 0.1229  0.0246 -0.1111  0.0850  0.1644 -0.1856
[torch.DoubleTensor of size 3x6]



In [42]:
-- we can forget our history, though, by calling :forget()
seqLSTM:forget()
print(rememberLSTM:forward(songDataT)[5]) -- printing out just the final time-step

 0.0631 -0.1597 -0.0998  0.0124  0.0474 -0.0257
-0.0005 -0.0683 -0.0124  0.0068 -0.0066 -0.0748
 0.1229  0.0247 -0.0984  0.0828  0.1579 -0.1836
[torch.DoubleTensor of size 3x6]



In [43]:
-- if we use :remember('neither') or :remember('eval'), :forget() is called internally before each :forward()
seqLSTM:remember('neither')
print(rememberLSTM:forward(songDataT)[5]) -- printing out just the final time-step
-- now it doesn't change if we call forward twice
print(rememberLSTM:forward(songDataT)[5]) -- printing out just the final time-step

 0.0631 -0.1597 -0.0998  0.0124  0.0474 -0.0257
-0.0005 -0.0683 -0.0124  0.0068 -0.0066 -0.0748
 0.1229  0.0247 -0.0984  0.0828  0.1579 -0.1836
[torch.DoubleTensor of size 3x6]

 0.0631 -0.1597 -0.0998  0.0124  0.0474 -0.0257
-0.0005 -0.0683 -0.0124  0.0068 -0.0066 -0.0748
 0.1229  0.0247 -0.0984  0.0828  0.1579 -0.1836
[torch.DoubleTensor of size 3x6]



## FastLSTM

-- just mention FastLSTM and that it doesn't have peep-hole connections

## Transducer RNNs
   -- Just mention SequencerCriterions